In [1]:
import openfl.native as fx
from openfl.federated import FederatedModel, FederatedDataSet

2023-04-10 18:53:48.526845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 18:53:48.655277: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-10 18:53:48.678558: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-10 18:53:49.071651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Load Dataset and Transfer 
### Define Torch Dataloader

In [2]:
import torch
import torchvision
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## Replace here with path of your folder. 
dataset = torchvision.datasets.ImageFolder(root='/home/avinash123/Desktop/Spring 2023/CS6675/FinalProject/archive/gaussian_filtered_images/gaussian_filtered_images', transform=transform)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

### Display data into train and valid sets

In [3]:
import matplotlib.pyplot as plt
import numpy as np

def index_to_class_name(index, dataset):
    return dataset.classes[index]

len_images = 10
images, labels = next(iter(dataloader))
sample_indices = np.random.choice(len(images), len_images, replace=False)
sample_images = images[sample_indices]
sample_labels = labels[sample_indices]

for i, image in enumerate(sample_images):
    class_name = index_to_class_name(sample_labels[i], dataset)
    plt.subplot(2, 5, i+1)
    plt.imshow(image.permute(1, 2, 0))
    plt.title(class_name)
    plt.axis('off')

plt.show() 

NameError: name 'dataloader' is not defined

In [3]:
import matplotlib.pyplot as plt
import numpy as np

### Distribute data into train and valid data sets


In [4]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset), shuffle=False)
data, labels = next(iter(data_loader))
print("The data shape is {}".format(data.shape))

The data shape is torch.Size([3662, 3, 224, 224])


In [5]:
import torch.nn.functional as F
def split_and_return_train_valid(data, labels, split = 0.2):
  test_indices = np.random.rand(data.shape[0]) < split
  train_indices = ~test_indices
  train_data, test_data = data[train_indices], data[test_indices]
  train_labels, test_labels = labels[train_indices], labels[test_indices]
  train_labels, test_labels = F.one_hot(train_labels), F.one_hot(test_labels)
  return train_data, train_labels, test_data, test_labels

train_data, train_labels, test_data, test_labels = split_and_return_train_valid(data, labels) 
print("Train Data Length is {}".format(len(train_data)))
print("Test Data Length is {}".format(len(test_data)))

Train Data Length is 2947
Test Data Length is 715


In [6]:
from openfl.federated import FederatedModel,FederatedDataSet
batch_size = 16
num_classes = 5
fl_data = FederatedDataSet(train_data, train_labels, test_data, test_labels, 
                           batch_size = batch_size , num_classes = num_classes, 
                           train_size = RandomNumPyDataSplitter)

###  Define Model - Pretrained Model
We can use a pretrained model like Inception or VGG Net to perform classification. These have already been pretrained on large-scale image data and we need to train only some final linear layers.

In [7]:

import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

class InceptionV1(nn.Module):
    def __init__(self, num_classes=5):
        super(InceptionV1, self).__init__()
        self.num_classes = num_classes
        self.inception = models.googlenet(pretrained=True)
        
        for param in self.inception.parameters():
            param.requires_grad = False
            
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)

    def forward(self, x):
        x = self.inception(x)
        return F.log_softmax(x, dim=1)
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.binary_cross_entropy_with_logits(input=output,target=target.float())


In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(InceptionV1())

## Earlier Parameters: 51386181

/home/avinash123/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/avinash123/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


5125

In [9]:
net = InceptionV1()
net(data[3:8]).shape

torch.Size([5, 5])

In [10]:
fl_model = FederatedModel(build_model=InceptionV1, optimizer=optimizer, loss_fn=cross_entropy, data_loader=fl_data)

tried to remove tensor: __opt_state_needed not present in the tensor dict


### Define OpenFlow Plan

In [11]:
NUM_COLLABORATORS = 5
collaborator_list = [str(i) for i in range(NUM_COLLABORATORS)]
experiment_collaborators = {col_name:col_model for col_name, col_model \
                                 in zip(collaborator_list, fl_model.setup(len(collaborator_list)))}

tried to remove tensor: __opt_state_needed not present in the tensor dict
tried to remove tensor: __opt_state_needed not present in the tensor dict
tried to remove tensor: __opt_state_needed not present in the tensor dict
tried to remove tensor: __opt_state_needed not present in the tensor dict
tried to remove tensor: __opt_state_needed not present in the tensor dict


In [12]:
fx.init('torch_cnn_mnist', col_names=collaborator_list)

Creating Workspace Directories
Creating Workspace Templates



New workspace directory structure:
workspace
├── save
│   ├── torch_cnn_mnist_last.pbuf
│   ├── torch_cnn_mnist_init.pbuf
│   └── torch_cnn_mnist_best.pbuf
├── src
│   ├── ptmnist_inmemory.py
│   ├── tfmnist_inmemory.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-39.pyc
│   │   └── mnist_utils.cpython-39.pyc
│   ├── pt_cnn.py
│   └── keras_cnn.py
├── agg_to_col_two_signed_cert.zip
├── agg_to_col_one_signed_cert.zip
├── agg_to_col_3_signed_cert.zip
├── agg_to_col_1_signed_cert.zip
├── agg_to_col_4_signed_cert.zip
├── agg_to_col_0_signed_cert.zip
├── data
├── requirements.txt
├── agg_to_col_2_signed_cert.zip
├── plan
│   ├── defaults
│   ├── cols.yaml
│   ├── data.yaml
│   └── plan.yaml
├── logs
│   └── cnn_mnist
│       ├── events.out.tfevents.1681162787.aviLaptop.21616.0
│       ├── events.out.tfevents.1681166465.aviLaptop.29267.0
│       ├── events.out.tfevents.1681166403.aviLaptop.29194.0
│       ├── events.out.tfevents.1681166330.aviL

In [ ]:
final_fl_model = fx.run_experiment(experiment_collaborators, override_config={"aggregator.settings.rounds_to_train": 20})

[18:55:16] INFO     Updating aggregator.settings.rounds_to_train to 20...                                                                          ]8;id=750252;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/native/native.py\native.py]8;;\:]8;id=694932;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/native/native.py#102\102]8;;\

           INFO     FL-Plan hash is 130b3e0614caa1414f180935e414149febc49b5d6d9ed0eb94f8bddb33780a4523e5deded8710226a18bce74e5d6895d                 ]8;id=740715;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=422513;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/plan/plan.py#235\235]8;;\

           INFO     Building 🡆 Object NoCompressionPipeline from openfl.pipelines Module.                                                            ]8;id=732101;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=174613;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/plan/plan.py#171\171]8;;\

[18:55:17] INFO     Creating Initial Weights File    🠆 save/torch_cnn_mnist_init.pbuf                                                              ]8;id=276091;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/native/native.py\native.py]8;;\:]8;id=838993;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/native/native.py#277\277]8;;\

           INFO     Starting Experiment...                                                                                                         ]8;id=204728;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/native/native.py\native.py]8;;\:]8;id=147232;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/native/native.py#281\281]8;;\

...

           INFO     Collaborator 3 is sending task results for aggregated_model_validation, round 4                                            ]8;id=39641;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=301226;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#520\520]8;;\

           METRIC   Round 4, collaborator validate_agg aggregated_model_validation result acc:      0.517483                                   ]8;id=732121;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=532346;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/component/aggregator/aggregator.py#559\559]8;;\

[19:05:49] INFO     Using TaskRunner subclassing API                                                                                         ]8;id=20672;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=948564;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/component/collaborator/collaborator.py#253\253]8;;\

           INFO     Run 0 epoch of 4 round                                                                                                      ]8;id=671728;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/task/runner_pt.py\runner_pt.py]8;;\:]8;id=779030;file:///home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/task/runner_pt.py#155\155]8;;\

/home/avinash123/anaconda3/lib/python3.9/site-packages/openfl/federated/task/runner_pt.py:458: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


We get a model with validation score = 67% ( for 2 clients)
We get a model with validation score = 55% ( 3 epochs, 5 clients)
